In [1]:
import torch
import numpy as np
import pandas as pd
from pandas import DataFrame
from torch.autograd import Variable
import random
import math
import time
def rand_n_int(l, r, n):
    s=[]
    while(len(s)<n):
        x=random.randint(l,r)
        if x not in s:
            s.append(x)
    return s

random.seed(time.time())

data_arr = np.load("data_reduce.npy")
labels_pre = np.load("labels_full.npy")
labels_pre = labels_pre.reshape(-1,1)
cates = max(labels_pre)[0]+1
labels = []
for i in range(labels_pre.shape[0]):
    row = np.zeros(cates).tolist()
    row[labels_pre[i][0]] = 1
    labels.append(row)
    
labels = np.array(labels)    

test_index = np.array(rand_n_int(0, data_arr.shape[0]-1, math.floor(data_arr.shape[0]*0.3)))
train_index=[]
for i in range(data_arr.shape[0]):
    if i not in test_index:
        train_index.append(i)
train_index = np.array(train_index)
data_train = data_arr[train_index,:]
label_train = labels[train_index,:]
data_test = data_arr[test_index,:]
label_test = labels[test_index,:]

x, y = Variable(torch.FloatTensor(data_train)), Variable(torch.FloatTensor(label_train))
x_test, y_test = Variable(torch.FloatTensor(data_test)), Variable(torch.FloatTensor(label_test))

In [2]:
torch.manual_seed(time.time())
seq_net = torch.nn.Sequential(
    torch.nn.Linear(data_arr.shape[1], math.ceil(math.sqrt(data_arr.shape[1]*labels.shape[1]))),
    torch.nn.Sigmoid(),
    #torch.nn.Linear(math.ceil(math.sqrt(data_arr.shape[1]*labels.shape[1])), math.ceil(math.sqrt(data_arr.shape[1]*labels.shape[1]))),
    #torch.nn.Sigmoid(),
    torch.nn.Linear(math.ceil(math.sqrt(data_arr.shape[1]*labels.shape[1])), labels.shape[1]),
    torch.nn.Sigmoid()
)

In [3]:
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(seq_net.parameters(), lr=0.001, weight_decay=1e-5)
for t in range(2000):
    prediction = seq_net(x)                                      # 用网络预测一下
    loss = loss_func(prediction, y)                          # 计算损失
    optimizer.zero_grad()                                    # 清除上一步的梯度
    loss.backward()                                          # 反向传播, 计算梯度
    optimizer.step()                                         # 优化一步
    if t % 100 == 0:
        print('No.%d iteration. Loss=%.4f' % (t, loss.item()))

No.0 iteration. Loss=0.2577
No.100 iteration. Loss=0.0058
No.200 iteration. Loss=0.0034
No.300 iteration. Loss=0.0024
No.400 iteration. Loss=0.0019
No.500 iteration. Loss=0.0016
No.600 iteration. Loss=0.0014
No.700 iteration. Loss=0.0013
No.800 iteration. Loss=0.0011
No.900 iteration. Loss=0.0011
No.1000 iteration. Loss=0.0010
No.1100 iteration. Loss=0.0009
No.1200 iteration. Loss=0.0009
No.1300 iteration. Loss=0.0008
No.1400 iteration. Loss=0.0008
No.1500 iteration. Loss=0.0008
No.1600 iteration. Loss=0.0007
No.1700 iteration. Loss=0.0007
No.1800 iteration. Loss=0.0007
No.1900 iteration. Loss=0.0007


In [4]:
out = seq_net(x_test).detach().numpy()
out_arr = np.array([])
for i in range(out.shape[0]):
    out_arr = np.append(out_arr, [np.argmax(out[i])])
out_arr = out_arr.astype(int)
correct = 0
for i in range(label_test.shape[0]):
    if label_test[i][out_arr[i]] == 1:
        correct+=1
print("Accuracy: %.4f" % (correct/label_test.shape[0]))

Accuracy: 0.9248
